In [27]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw

In [28]:
geo = SplineGeometry()
m = 0.6
geo.AddCircle(c=(0, 0), r=1, bc="cyl", maxh=m) # 0.04
mesh = Mesh(geo.GenerateMesh(maxh=m))
mesh2 = Mesh(geo.GenerateMesh(maxh=m))
mesh.Curve(3);
mesh2.Curve(3)
scene1 = Draw(mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': None…

In [29]:
# vector space for shape gradient
VEC = H1(mesh, order=1, dim=2)
# Test and trial functions
PHI, X = VEC.TnT()
gfX = GridFunction(VEC)

In [30]:
surf_t = CoefficientFunction(1)
surf_0 = Integrate(surf_t,mesh)

dJOmega  = LinearForm(VEC)

# volume stuff
surf_t = CoefficientFunction(1)
surf_0 = Integrate(surf_t,mesh)

# bc prep
bc_tx = CoefficientFunction(x)
bc_ty = CoefficientFunction(y)
bc_0x = 1/surf_0*Integrate(bc_tx,mesh)
bc_0y = 1/surf_0*Integrate(bc_ty,mesh)

# bc constraints
beta0 = 1e-3
beta = Parameter(beta0)
bc_x = Parameter(1)
bc_x.Set((1/surf_0)*Integrate(bc_tx,mesh))

dJOmega += ((beta*((bc_x-bc_0x)**2)*dx)).DiffShape(X)

In [31]:
dJOmega.Assemble()
print("Initial bc: ", bc_x.Get())
print(dJOmega.vec[0:5])

Initial bc:  -2.4295022466382792e-17
       0
      0

       0
      0

       0
      0

       0
      0

       0
      0




In [37]:
mesh.UnsetDeformation()
test = CoefficientFunction((2*x+1,0.5*x))
gfX.Set(test)
mesh.SetDeformation(gfX)
scene = Draw(mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': None…

In [38]:
bc_x.Set((1/surf_0)*Integrate(bc_tx,mesh))
dJOmega.Assemble()
print(bc_x.Get())
print(abs(bc_0x-bc_x.Get()))
print(dJOmega.vec)

3.0085983712937034
3.0085983712937034
  0.00229899
 -0.0137204

  0.00452441
 6.61141e-18

  -0.00229899
 0.0137204

  -0.00452441
 -4.85588e-18

  0.00435703
 -0.0121723

  0.00523175
 -0.00699633

  0.00289872
 0.00699633

  0.000303506
 0.0121723

  -0.00435703
 0.0121723

  -0.00523175
 0.00699633

  -0.00289872
 -0.00699633

  -0.000303506
 -0.0121723

  -8.67362e-19
 2.60209e-18

  -1.73472e-18
 3.46945e-18

  4.33681e-19
 -3.37458e-18

  1.16552e-18
      0

  -8.67362e-19
 -1.81604e-18

  -3.25261e-19
      0

  1.73472e-18
      0




## try to do this as shape optimization

In [39]:
gfset = GridFunction(VEC)

b = BilinearForm(VEC)
b += (InnerProduct(grad(X),grad(PHI))).Compile() *dx + (InnerProduct(X,PHI)).Compile()*dx

def SolveDeformationEquation():
    rhs = gfX.vec.CreateVector()
    rhs.data = dJOmega.vec - b.mat * gfX.vec
    update = gfX.vec.CreateVector()
    update.data = b.mat.Inverse(VEC.FreeDofs()) * rhs
    gfX.vec.data += update

In [40]:
bc_x.Get()

3.0085983712937034

In [41]:
gfset.Set((0,0))
mesh.SetDeformation(gfset)
scene.Redraw()
iter_max = 1

for i in range(0,iter_max):
    mesh.SetDeformation(gfset)
    scene.Redraw()
    
    b.Assemble()
    bc_x.Set((1/surf_0)*Integrate(bc_tx,mesh)) # needed for multiple iterations (already done once before)
    dJOmega.Assemble()
    SolveDeformationEquation()
    
    mesh.UnsetDeformation()
    
    gfxnorm = Norm(gfX.vec)
    scale = 0.01 / gfxnorm
    gfset.vec.data -= scale * gfX.vec
    Redraw(blocking=True)

In [42]:
bc_x.Get()

-2.4295022466382792e-17